### Goal: Try again on estimation on region
Based on all the features we get this time.

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.metrics import mean_squared_error;

import datetime;

2.6.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [4]:
MONTHS = 60;
SPLIT = 48; # 2015-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 24;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/full_dataset_engineered.csv', index_col=0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "year-month"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,total_pop,percent_rental_units_occupied,percent_highschool,percent_work_from_home,percent_buildings_50+_units,median_building_age,median_age,income_per_capita,percent_native_am,percent_buildings_10_19_units,poverty_rate,percent_white,percent_commute_public_transport,percent_associates,percent_units_owner_occupied,percent_workforce_unemployed,percent_buildings_20_49_units,bs_total_stations,trader joe's,flood,gun_searches,pronouns,twitter,lululemon,retirement,same sex marriage,hurricane,political correctness,job opportunities,unemployment,layoff,wildfire,hashtag,fake news,multifamily_for_rent_searches,thrift,make america great again,flight tickets,black lives matter,lgbt,whole foods,fire,apartment_for_rent_searches,places to go,starbucks,getaway,euthanasia,net_job_rate,startup_firms,state_local_perc
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,946.0,21846.25,0.942265,0.383459,0.014618,0.049514,62.333333,36.508333,20935.75,0.000113,0.030202,0.199214,0.714892,0.022609,0.257328,0.423344,0.105077,0.036863,0.0,-0.662346,0.944722,-3.278835,-1.326593,1.004233,-0.212701,-1.425905,-0.588436,-0.285509,0.0,0.096863,3.466751,-0.823717,-0.443099,0.448996,-0.659443,-0.259098,-0.377642,-0.514772,-1.378408,-0.509999,-0.141562,0.291193,1.162014,-4.544528,-1.304734,-1.712329,-0.468627,-0.150792,1.993000,383.0,0.176793
1013,950.0,21870.50,0.942497,0.385186,0.014466,0.049557,62.666667,36.416667,20931.50,0.000226,0.030477,0.196885,0.715367,0.023198,0.255708,0.421607,0.105835,0.036368,0.0,0.435704,-1.199445,1.929184,-1.326593,1.303017,0.157345,-1.217118,-0.588436,-0.431227,0.0,1.061035,1.934232,-0.823717,-0.443099,0.448996,-0.659443,-0.765415,-0.826044,-0.514772,-0.847070,-0.509999,-1.071717,0.719432,0.225152,-0.351858,-0.015282,-1.241616,0.019798,-0.150792,1.750833,382.0,0.176313
1013,958.0,21894.75,0.942730,0.386912,0.014314,0.049601,63.000000,36.325000,20927.25,0.000339,0.030752,0.194555,0.715842,0.023787,0.254088,0.419870,0.106592,0.035873,0.0,-0.662346,-0.127361,0.951713,0.732224,1.751194,-0.952793,0.348785,-0.086484,-0.285509,0.0,-0.385223,0.991144,-0.823717,-0.443099,0.448996,0.570434,-0.336470,-0.527109,-0.514772,-0.847070,-0.509999,-1.536795,0.230016,1.295851,-2.620528,-1.304734,-1.268533,-0.794244,1.382217,1.508667,381.0,0.175832
1013,958.0,21919.00,0.942962,0.388639,0.014162,0.049644,63.333333,36.233333,20923.00,0.000452,0.031027,0.192226,0.716318,0.024375,0.252468,0.418133,0.107350,0.035378,0.0,1.533754,-0.127361,-0.167832,-1.326593,2.348763,-0.582747,-0.799544,-0.588436,-0.358368,0.0,-1.349394,1.462688,-0.823717,-0.443099,0.448996,-0.659443,-0.765415,-0.377642,-0.514772,-0.847070,-0.509999,-1.071717,1.086494,0.760502,-2.839611,-1.820515,-0.743268,-1.282668,-0.150792,1.266500,380.0,0.175352
1013,955.0,21943.25,0.943195,0.390365,0.014010,0.049687,63.666667,36.141667,20918.75,0.000565,0.031302,0.189897,0.716793,0.024964,0.250848,0.416396,0.108107,0.034883,0.0,-0.296329,0.944722,0.725700,-1.326593,1.153625,-0.336049,0.661965,-0.086484,-0.285509,0.0,0.578949,1.875289,-0.823717,3.266281,-1.393660,0.570434,-1.673442,-0.975511,-0.514772,-0.581401,-0.509999,0.788593,-0.198223,1.162014,-3.858335,0.500500,-0.248671,-1.119860,-1.172798,1.024333,379.0,0.174872


In [6]:
FEATURES = multi_data.shape[1] - 1;

feature_name = list(multi_data.columns);

In [7]:
# In our first try, just look at the zip codes in NY. zip:10001-14905
multi_zip = zip_ids;
# list(multi_data[(multi_data.index >= 10001) & (multi_data.index <= 14905)].index.unique());
print(len(multi_zip))

1301


### Utility functions

In [8]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size);
    return dataset;

In [9]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [10]:
def NN_dataprep(multi):
    cities_stats = {};
    dataset = windowed_dataset(np.zeros((1, FEATURES+1)), WINDOW_SIZE, BATCH_SIZE, 60);
    validset = windowed_dataset(np.zeros((1, FEATURES+1)), WINDOW_SIZE, BATCH_SIZE, 60);

    #for zip_num in multi_data["zip"].unique():
    for zip_num in multi:
        test = multi_data[multi_data.index == zip_num];
        single_city_series = np.array(test);

        scaler = MinMaxScaler();
        
        single_city_transformed = scaler.fit_transform(single_city_series)
        
        cities_stats[zip_num] = {"min": scaler.data_min_[0],
                                 "range": scaler.data_range_[0]};

        single_city_train = single_city_transformed[:SPLIT];
        single_city_test = single_city_transformed[SPLIT-WINDOW_SIZE:];

        cityset = windowed_dataset(single_city_train, WINDOW_SIZE, BATCH_SIZE, 60);
        cityvalid = windowed_dataset(single_city_test, WINDOW_SIZE, BATCH_SIZE, 60);

        dataset = dataset.concatenate(cityset);
        validset = validset.concatenate(cityvalid);
        
    dataset = dataset.prefetch(1);
    validset = validset.prefetch(1);
        
    return dataset, validset, cities_stats;

In [11]:
def NN_model(dataset, termination=0, test=None):
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    class myCallbacks(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            mse = logs.get("mse");
            if(mse < termination):
                print("\nGot an mse at {:.4f} in epoch {} and stopped training\n".format(mse, epoch));
                self.model.stop_training = True;
            
    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.LSTM(32, return_sequences=False,
                             input_shape = [WINDOW_SIZE, FEATURES+1]),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.2),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,WINDOW_SIZE,FEATURES+1))
    model.summary()

    if not test: history = model.fit(dataset, epochs=20, callbacks = [callback], verbose = 1);
    else: history = model.fit(dataset, epochs=20, validation_data=test,\
                              callbacks=[callback], verbose = 1);
    return model;

In [12]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    actual = list(series_transformed[WINDOW_SIZE:, 0]);

    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

In [13]:
@tf.autograph.experimental.do_not_convert
def NN_group_test(multi, termination=0, plot=False):
    '''
    Input: A list of zip codes
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    dataset_train, dataset_test, city_stats = NN_dataprep(multi);
        
#     for x, y in dataset_train:
#         print(np.array(x).shape, np.array(y).shape);

    model = NN_model(dataset_train, termination, dataset_test);
    model.save('.//saved_models/NN_model_{0:%Y-%m-%d %H-%M-%S}.h5'.format(datetime.datetime.now()));
    
    score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[], "forecast":[]};

    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    time_actual = range(WINDOW_SIZE, MONTHS);
    
    for zip_num in multi:
        scaler = MinMaxScaler();
        series = np.array(multi_data[multi_data.index == zip_num]);
        series_transformed = scaler.fit_transform(series);
    
        # Forecasting
        results, actual, pure_forecast = NN_forecast(model, series_transformed);

        # Compute MSE
        M_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.data_range_[0];
        M_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.data_range_[0];
        M_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.data_range_[0];
    
        if plot: # If the plot option is selected, plot the graph.
            plt.figure(figsize=(10, 6))
            plot_series(time_actual, np.array(actual)*scaler.data_range_[0]+scaler.data_min_[0]);
            plot_series(time_actual, np.array(results)*scaler.data_range_[0]+scaler.data_min_[0]);
            plot_series(time_test, np.array(pure_forecast)*scaler.data_range_[0]+scaler.data_min_[0]);
            plt.show();

        score_dict["zip"].append(zip_num);
        score_dict["RMSE_train"].append(M_train);
        score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(pure_forecast);
        print(zip_num, M_train, M_test, M_pure)

    return score_dict;

### The script

In [ ]:
score_dict = NN_group_test(multi_zip, 0.004, True);

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                10752     
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 11,297
Trainable params: 11,297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
score_dict;

In [ ]:
print(sum(score_dict["RMSE_test"])/len(score_dict["RMSE_test"]), sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

In [ ]:
temp = score_dict.copy();

In [ ]:
from sklearn.metrics import r2_score;
r2 = [];
actual = [];
predicted = [];
for i in range(len(multi_zip)):
    scaler = MinMaxScaler();
    frame = np.array(multi_data[multi_data.index == multi_zip[i]][["zri"]]);
    frame = scaler.fit_transform(frame);

    actual.extend(list(np.array(multi_data[multi_data.index == temp['zip'][i]]["zri"])[-TEST_LENGTH:]));
    predicted.extend(list(temp['forecast'][i]*scaler.data_range_[0]+scaler.data_min_[0]));

    # r2_zip = mean_squared_error(np.array(multi_data[multi_data.index == temp['zip'][i]]["zri"])[-TEST_LENGTH:], temp['forecast'][i]*scaler.data_range_[0]+scaler.data_min_[0]);
    # print(r2_zip, np.array(multi_data[multi_data.index == temp['zip'][i]]["zri"])[-TEST_LENGTH:], temp['forecast'][i]*scaler.data_range_[0]+scaler.data_min_[0]);
    # r2.append(r2_zip);

In [ ]:
r2_score(actual, predicted)

In [ ]:
test = pd.DataFrame(score_dict);
test.to_csv('.//Results in CSV files/ZRI_and_engineered.csv', index = False);